In [95]:
import plotly.express as plt 
import pandas as pd
import subprocess
import os
import io
import numpy as np
import sys 
import platform
sys.path.append('./modules/')
from mva import *
import simulation
import plotly.io as pio                                                        
pio.kaleido.scope.chromium_args += ("--single-process",) 

In [96]:
matrix = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0.004,0.006,0.9,0.065,0.025],
    [0,0,1,0,0],
    [0,0,1,0,0]],np.dtype('d'))

In [97]:
v = np.zeros(len(matrix))
v[0] = 1
for i in range(1000):
    v = v @ matrix
    pass
v
for i in range(1,len(v)):
    v[i] = v[i]*(1/v[0])
    pass
v[0] = 1
v

array([  1.  ,   2.5 , 250.  ,  16.25,   6.25])

In [98]:
mva = MVA(matrix,[5000,210,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)

def mvaToDataframe(matrix: np.ndarray)-> pd.DataFrame:
    vals = pd.DataFrame()
    vals["DELAY"] = matrix[0]
    vals["SWAP_IN"] = matrix[1]
    vals["CPU"] = matrix[2]
    vals["IO1"] = matrix[3]
    vals["IO2"] = matrix[4]
    vals["N"] = [i for i in range(0,len(matrix[0]))]
    return vals

mva()

In [99]:
#utilization = pd.read_csv("../../build/Scheduler/utilizations.csv",sep=";")
#throughputs = pd.read_csv("../../build/Scheduler/throughputs.csv", sep=";")
#meanWaits = pd.read_csv("../../build/Scheduler/meanWaits.csv",sep=";")
#meanClients = pd.read_csv("../../build/Scheduler/meanClients.csv", sep=";")

utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

visits = {}
visits["CPU"] = 250
visits["SWAP_IN"] = 2.5
visits["IO1"] = 16.25
visits["IO2"] = 6.25 

In [100]:
print(meanWaits)

     DELAY     SWAP_IN       CPU        IO1          IO2   N
0      0.0    0.000000  0.000000   0.000000     0.000000   0
1   5000.0  210.000000  2.700000  40.000000   180.000000   1
2   5000.0  223.824451  2.928527  43.260188   205.391850   2
3   5000.0  238.389124  3.177571  46.793396   235.823993   3
4   5000.0  253.533652  3.446068  50.579796   272.282610   4
5   5000.0  269.032970  3.731638  54.581095   315.853912   5
6   5000.0  284.600954  4.030402  58.738515   367.680155   6
7   5000.0  299.904587  4.336961  62.973254   428.890837   7
8   5000.0  314.589239  4.644616  67.190305   500.511697   8
9   5000.0  328.312511  4.945852  71.285815   583.361328   9
10  5000.0  340.780878  5.233042  75.157177   677.952217  10
11  5000.0  351.781406  5.499257  78.714047   784.416577  11
12  5000.0  361.201359  5.739004  81.887950   902.474954  12
13  5000.0  369.031668  5.948742  84.638366  1031.456793  13
14  5000.0  375.354754  6.127068  86.954192  1170.369616  14
15  5000.0  380.321190  

In [101]:


activeTimes = pd.DataFrame()
activeTimes["N"] = meanWaits["N"]
activeTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
activeTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
activeTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
activeTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]
print(activeTimes)


     N          CPU     SWAP_IN          IO1           IO2
0    0     0.000000    0.000000     0.000000      0.000000
1    1   675.000000  525.000000   650.000000   1125.000000
2    2   732.131662  559.561129   702.978057   1283.699060
3    3   794.392798  595.972811   760.392684   1473.899954
4    4   861.517004  633.834130   821.921680   1701.766314
5    5   932.909505  672.582426   886.942799   1974.086948
6    6  1007.600472  711.502385   954.500871   2298.000967
7    7  1084.240242  749.761468  1023.315381   2680.567734
8    8  1161.154028  786.473098  1091.842463   3128.198104
9    9  1236.462960  820.781277  1158.394499   3646.008298
10  10  1308.260497  851.952195  1221.304120   4237.201358
11  11  1374.814216  879.453516  1279.103264   4902.603606
12  12  1434.751032  903.003398  1330.679193   5640.468464
13  13  1487.185540  922.579169  1375.373453   6446.604955
14  14  1531.766887  938.386884  1413.005626   7314.810103
15  15  1568.642694  950.802975  1443.824881   8237.5124

In [102]:
print(meanClients)

       DELAY   SWAP_IN       CPU       IO1        IO2   N
0   0.000000  0.000000  0.000000  0.000000   0.000000   0
1   0.626959  0.065831  0.084639  0.081505   0.141066   1
2   1.207967  0.135186  0.176878  0.169835   0.310133   2
3   1.739199  0.207303  0.276321  0.264495   0.512681   3
4   2.217531  0.281109  0.382088  0.364527   0.754744   4
5   2.640886  0.355243  0.492741  0.468463   1.042668   5
6   3.008543  0.428117  0.606282  0.574331   1.382727   6
7   3.321350  0.498044  0.720228  0.679758   1.780621   7
8   3.581768  0.563393  0.831797  0.782145   2.240896   8
9   3.793740  0.622766  0.938164  0.878929   2.766401   9
10  3.962368  0.675150  1.036762  0.967851   3.357870  10
11  4.093488  0.720006  1.125557  1.047199   4.013750  11
12  4.193194  0.757294  1.203238  1.115959   4.730316  12
13  4.267404  0.787404  1.269284  1.173855   5.502053  13
14  4.321529  0.811053  1.323915  1.221269   6.322233  14
15  4.360267  0.829151  1.367940  1.259092   7.183550  15
16  4.387512  

In [103]:
def produce_graph_and_save(dataframe: pd.DataFrame, name:str, yaxis:str, log:bool = False):
    fig = plt.line(dataframe,y=["CPU","SWAP_IN","IO1","IO2"], x="N",title=name,log_y=log)
    fig.update_yaxes(title_text=yaxis)
    data = fig.to_image("png",engine='kaleido')
    f = open("./Images/{}.png".format(name),"+wb")
    f.write(data)
    f.close()
    fig.show()
pass

In [104]:
produce_graph_and_save(utilization,"Utilizations","B/T")

In [105]:
produce_graph_and_save(meanWaits,"meanWaits","ms",True)

In [106]:
produce_graph_and_save(meanClients,"meanClients","n")

In [107]:
produce_graph_and_save(throughputs,"throughputs","C/T")

In [108]:
produce_graph_and_save(activeTimes,"active_times","ms",True)

In [109]:
totalActiveTime = pd.DataFrame()
totalActiveTime["N"] = activeTimes["N"]
totalActiveTime["ActiveTime"] = activeTimes["CPU"] + activeTimes["IO1"] + activeTimes["IO2"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

In [110]:
totalActiveTime["ActiveTime"] = (meanClients["CPU"] + meanClients["IO1"] + meanClients["IO2"])/throughputs["SWAP_IN"]
plt.line(totalActiveTime,x="N",y="ActiveTime")


# Markov chain model

In [111]:
mva = MVA(matrix,[5000,0,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)
mva()
utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)

activeTimes = pd.DataFrame()
activeTimes["N"] = meanWaits["N"]
activeTimes["CPU"] = meanWaits["CPU"]*visits["CPU"]
activeTimes["SWAP_IN"] = meanWaits["SWAP_IN"]*visits["SWAP_IN"]
activeTimes["IO1"] = meanWaits["IO1"]*visits["IO1"]
activeTimes["IO2"] =meanWaits["IO2"]*visits["IO2"]


In [112]:
produce_graph_and_save(utilization,"Utilizations_markov","B/T")

In [113]:
produce_graph_and_save(meanWaits,"meanWaits_markov","ms",True)

In [114]:
produce_graph_and_save(meanClients,"meanClients_markov","n")

In [115]:
produce_graph_and_save(throughputs,"throughputs_markov","C/T")

In [116]:
totalActiveTime = pd.DataFrame()
totalActiveTime["N"] = activeTimes["N"]
totalActiveTime["ActiveTime"] = activeTimes["CPU"] + activeTimes["IO1"] + activeTimes["IO2"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

In [117]:
totalActiveTime["ActiveTime"] = (meanClients["CPU"] + meanClients["IO1"] + meanClients["IO2"])/throughputs["SWAP_IN"]
plt.line(totalActiveTime,x="N",y="ActiveTime")

# Simulative model

In [118]:
simulator = simulation.SimulatorCommander()


In [119]:
s20 = simulator.fetch_data(10,"Simplified_N20",123456789)

In [120]:
copy = s20
hitted = ((copy["LB"] < copy["Expected"]) & (copy["Expected"] < copy["HB"]))
copy["Hitted"] = hitted

In [121]:
def hitpercentage(data:pd.DataFrame,station,measure):
    view = data[(data["Station"] == station)& (data["Measure"] == measure)]
    return (view["Hitted"].sum()/10)*100
    pass

for station in simulator.stationToRecord:
    print("Station{} Measure {} HitPercentage {}".format(station,"meanwaits",hitpercentage(copy,station,"meanwaits")))
    print("Station{} Measure {} HitPercentage {}".format(station,"meanclients",hitpercentage(copy,station,"meanclients")))
    pass

StationCPU Measure meanwaits HitPercentage 100.0
StationCPU Measure meanclients HitPercentage 100.0
StationIO1 Measure meanwaits HitPercentage 90.0
StationIO1 Measure meanclients HitPercentage 90.0
StationIO2 Measure meanwaits HitPercentage 90.0
StationIO2 Measure meanclients HitPercentage 90.0
StationSWAP_IN Measure meanwaits HitPercentage 90.0
StationSWAP_IN Measure meanclients HitPercentage 90.0


In [122]:
activetimes = copy[copy["Station"] == "ActiveTime"]
hits = activetimes["Hitted"].sum()
print("Activetimes hitted {}%".format(hits/10*100))

Activetimes hitted 90.0%


In [123]:
Default = simulator.fetch_data(10,"Default")
NegExp = simulator.fetch_data(10,"NegExpCpu")
LTCpu = simulator.fetch_data(10,"LTCpu")
NegExpLt = simulator.fetch_data(10,"NegExpLt")





Default case

In [124]:
print(Default)

       Station      Measure            R           LB           HB  Samples  \
0          CPU  meanclients     1.368781     1.337820     1.399742       40   
1          CPU    meanwaits     5.940569     5.845767     6.035371       40   
2          IO1  meanclients     1.256253     1.230519     1.281986       40   
3          IO1    meanwaits    88.118018    86.847111    89.388924       40   
4          IO2  meanclients     6.539749     6.492380     6.587118       40   
..         ...          ...          ...          ...          ...      ...   
85         IO2  meanclients     6.532628     6.493150     6.572106       40   
86         IO2    meanwaits  1194.580389  1183.062655  1206.098124       40   
87     SWAP_IN  meanclients     0.827107     0.807667     0.846547       40   
88     SWAP_IN    meanwaits   378.132436   371.553149   384.711724       40   
89  ActiveTime    cycleTime  4186.637650  4056.929625  4316.345676       40   

    Precision     Expected  Simulation       Seed  

In [125]:
def print_active_time(data: pd.DataFrame, scenario, labels:[str]  =["LB","HB","R"]):
    view = data[data["Station"] == "ActiveTime"]
    fig= plt.line(view,x="Simulation", y=labels,title="{} Scenario: Mean Active Time".format(scenario))
    fig.update_yaxes(title_text="Mean Active Time(ms)")
    data = fig.to_image("png")
    f = open("./Images/meanActiveTime_{}.png".format(scenario),"+wb")
    f.write(data)
    f.close()
    return fig
print_active_time(s20,"Simplified_N20", labels=["LB","HB","R","Expected"])

In [126]:
print_active_time(Default,"Default")

In [127]:
print_active_time(NegExp,"NegExp")

In [128]:
print_active_time(LTCpu,"LTCpu")

In [129]:
print_active_time(NegExpLt,"NegExpLt")

In [130]:
activeTimesR = pd.DataFrame()
def loadActiveTime(data:pd.DataFrame,scenario):
    activeTimesR[scenario] = data[data["Station"] == "ActiveTime"]["R"]
    pass

activeTimesR["Simulation"] = Default[Default["Station"] == "ActiveTime"]["Simulation"]
activeTimesR["Expected"] = Default[Default["Station"] == "ActiveTime"]["Expected"]

loadActiveTime(Default,"Default")
loadActiveTime(NegExp,"NegExp")
loadActiveTime(NegExpLt,"NegExpLt")
loadActiveTime(LTCpu,"LTCpu")
loadActiveTime(s20,"Simplified_N20")
fig = plt.line(activeTimesR,x="Simulation",y=[x for x in activeTimesR.columns if x != "Simulation" and x != "Expected"],title="Mean active Time for scenario")
fig.update_yaxes(title_text="Mean active time(ms)")
fig.show()

Analysis of CPU

In [131]:
def print_cpu_measure(data: pd.DataFrame, scenario, measure, labels = ["LB","HB","R"]):
    view = data[(data["Station"] == "CPU")& (data["Measure"] == measure)]
    fig = plt.line(view, x="Simulation" , y=labels, title="{} CPU scenario {}".format(measure,scenario))
    data = fig.to_image("png")
    f = open("./Images/{}_CPU_{}.png".format(scenario,measure),"+wb")
    f.write(data)
    f.close()
    return fig,view
    pass

fig,view = print_cpu_measure(s20,"Simplified_N20","meanwaits", ["LB","HB","R","Expected"])
fig.show()

In [132]:
print_cpu_measure(s20,"Simplified_N20","meanclients",["LB","HB","R","Expected"])[0].show()

In [133]:
print_cpu_measure(Default,"Default","meanclients")[0]

In [134]:
print_cpu_measure(Default,"Default","meanwaits")[0]

In [135]:
print_cpu_measure(LTCpu,"LTCpu","meanwaits")[0]

In [136]:
print_cpu_measure(LTCpu,"LTCpu","meanclients")[0]


In [137]:
print_cpu_measure(NegExp,"NexExp","meanwaits")[0]

In [138]:
print_cpu_measure(NegExp,"NexExp","meanclients")[0]


In [139]:
print_cpu_measure(NegExpLt,"NegExpLt","meanwaits")[0]

In [140]:
print_cpu_measure(NegExpLt,"NegExpLt","meanclients")[0]

In [141]:
overallMeanwaits = pd.DataFrame()
